In [1]:
# ===========================================
# KINDA EXPERIMENTAL - USED FOR EXPLORATION OF DATA - dont expect this to run as it is
# ===========================================

import pandas as pd
import numpy as np
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer, RegexpStemmer
from string import punctuation

from collections import Counter

In [79]:
porter = PorterStemmer()
wnl = WordNetLemmatizer() 
stop = stopwords.words('english')

stop.append("!")
stop.append(',')
stop.append('')
stop.append('=')
stop = list(stop)
re = RegexpStemmer('[0-9]+')

# Preprocess text
def preprocess_text(string):
    # Annoying things!
    string = string.replace("=", "")
    string = string.replace("-", "")
    string = string.replace("'", "")
    tokens = word_tokenize(str(string))
    # Punctuations
    tokens = [token for token in tokens if token not in punctuation]
    # Stopwords
    tokens = [token for token in tokens if token not in stop]
    # Numbers
    tokens = [re.stem(token) for token in tokens]
    # convert to lowercase
    tokens = [token.lower() for token in tokens]
    return (tokens)

# Make unigrams and bygrams from a classified data 
def make_grams(classified_data):
    unigram_frequencies = Counter([])
    #bigram_frequencies = Counter([])
    
    sentances = classified_data["comment_text"]
    for sentance in sentances:
        unigrams = ngrams(sentance, 1)
        # bigrams = ngrams(sentance, 2)
        unigram_frequencies += Counter(unigrams)
        # bigram_frequencies += Counter(bigrams)
    return unigram_frequencies# , bigram_frequencies)

In [60]:
#train = pd.read_csv('test.csv', index_col=0)
#train_labels = pd.read_csv('test_labels.csv', index_col=0)

# Process data!
#train["comment_text"] = train.comment_text.apply(preprocess_text)
#data = pd.concat([train, train_labels], axis=1)


data = pd.read_csv('train.csv', index_col=0)
data["comment_text"] = data.comment_text.apply(preprocess_text)

In [63]:
# Method that gets documents that only belong to only one of the classes
def filter_comments(data, toxic = 0, severe_toxic = 0, obscene = 0, threat = 0, insult = 0, identity_hate = 0):
    condition = (data.toxic == toxic) & (data.severe_toxic == severe_toxic) & (data.obscene == obscene) & (data.threat == threat) & (data.insult == insult) & (data.identity_hate == identity_hate)
    return data[condition]
    
    
toxic_only = filter_comments(data, toxic = 1)
severe_toxic_only = filter_comments(data, severe_toxic = 1)
obscene_only = filter_comments(data, obscene = 1)
threat_only = filter_comments(data, threat = 1)
insult_only = filter_comments(data, insult = 1)
identity_hate_only = filter_comments(data, identity_hate = 1)
cleant_documents = filter_comments(data)

In [64]:
# Method to print precenage of documents compared to larger subset
def partition(message, all_docs, subset_docs):
    return message + "\t" +  str(round(len(subset_docs) / len(all_docs), 6)) + "%\t ,count: " + str(len(subset_docs))


total_documents = len(data)
clean_document_count = len(cleant_documents)
toxic_document_count = len(toxic_only)
severe_toxic_document_count = len(severe_toxic_only)
obscene_document_count = len(obscene_only)
threat_document_count = len(threat_only)
insult_document_count = len(insult_only)
identity_hate_document_count = len(identity_hate_only)

print("Precentage of documents that belong to ONLY ONE class")
print("============")
print("Total documents: ", total_documents)
print(partition("Clean documents: \t", data, cleant_documents))
print("============")
print(partition("Only toxic documents: \t", data, toxic_only))
print(partition("Only severe toxic documents: ", data, severe_toxic_only))
print(partition("Only obscene documents: ", data, obscene_only))
print(partition("Only threath documents: ", data, threat_only))
print(partition("Only insult documents: \t", data, insult_only))
print(partition("Only identity documents: ", data, identity_hate_only))

Precentage of documents that belong to ONLY ONE class
Total documents:  159571
Clean documents: 		0.898321%	 ,count: 143346
Only toxic documents: 		0.035508%	 ,count: 5666
Only severe toxic documents: 	0.0%	 ,count: 0
Only obscene documents: 	0.001987%	 ,count: 317
Only threath documents: 	0.000138%	 ,count: 22
Only insult documents: 		0.001886%	 ,count: 301
Only identity documents: 	0.000338%	 ,count: 54


In [65]:
has_toxic = data[(data.toxic == 1)]
has_severe_toxic = data[(data.severe_toxic == 1)]
has_obscene = data[(data.obscene == 1)]
has_threat = data[(data.threat == 1)]
has_insult = data[(data.insult == 1)]
has_identity_hate = data[(data.identity_hate == 1)]


print("Precentage of documents that belong to a class")
print("============")
print("Total documents: ", total_documents)
print(partition("Clean documents: \t", data, cleant_documents))
print("============")
print(partition("Toxic documents: \t", data, has_toxic))
print(partition("Severe toxic documents: ", data, has_severe_toxic))
print(partition("Obscene documents \t: ", data, has_obscene))
print(partition("Threat documents: \t", data, has_threat))
print(partition("Insult documents: \t", data, has_insult))
print(partition("Identity hate documents: ", data, has_identity_hate))

( total_documents - clean_document_count ) / total_documents

Precentage of documents that belong to a class
Total documents:  159571
Clean documents: 		0.898321%	 ,count: 143346
Toxic documents: 		0.095844%	 ,count: 15294
Severe toxic documents: 	0.009996%	 ,count: 1595
Obscene documents 	: 	0.052948%	 ,count: 8449
Threat documents: 		0.002996%	 ,count: 478
Insult documents: 		0.049364%	 ,count: 7877
Identity hate documents: 	0.008805%	 ,count: 1405


0.10167887648758234

In [66]:
def filter_comments(data, condition):
    return data[condition]

classes = {}
classes["Toxic"] = (data.toxic == 1)
classes["SevereToxic"] = (data.severe_toxic == 1)
classes["Obscene"] = (data.obscene == 1)
classes["Threat"] = (data.threat == 1)
classes["Insult"] = (data.insult == 1)
classes["IdentityHate"] = (data.identity_hate == 1)

class_library = ["Toxic", "SevereToxic", "Obscene", "Threat", "Insult", "IdentityHate"]
freq_matrix = np.zeros((len(classes), len(classes)))
some_matrix = np.zeros((len(classes), len(classes)))

counter1 = 0
counter2 = 0
for class1 in class_library:
    counter2 = 0
    for class2 in class_library:
        condition = classes[class1] & classes[class2]
        common_elements = filter_comments(data, condition)
        freq_matrix[counter1,counter2] =  round(len(common_elements)  / (len(data) - len(cleant_documents)), 3)
        counter2 += 1
    counter1+= 1
        
counter1 = 0
counter2 = 0
for class1 in class_library:
    counter2 = 0
    class_elements = filter_comments(data, classes[class1])
    for class2 in class_library:
        condition2 = classes[class1] & classes[class2]
        common_elements = filter_comments(data, condition2)
        some_matrix[counter1,counter2] =  round(len(common_elements)  / (len(class_elements)), 3)
        counter2 += 1
    counter1+= 1
    

print("\n")
    
print(freq_matrix)

print("\n")

print(some_matrix)



[[0.943 0.098 0.489 0.028 0.453 0.08 ]
 [0.098 0.098 0.093 0.007 0.084 0.019]
 [0.489 0.093 0.521 0.019 0.379 0.064]
 [0.028 0.007 0.019 0.029 0.019 0.006]
 [0.453 0.084 0.379 0.019 0.485 0.071]
 [0.08  0.019 0.064 0.006 0.071 0.087]]


[[1.    0.104 0.518 0.029 0.48  0.085]
 [1.    1.    0.951 0.07  0.86  0.196]
 [0.938 0.18  1.    0.036 0.728 0.122]
 [0.939 0.234 0.63  1.    0.642 0.205]
 [0.932 0.174 0.781 0.039 1.    0.147]
 [0.927 0.223 0.735 0.07  0.826 1.   ]]


In [71]:
# Calculate how many words comments in certain class have
def calculate_avarage_words(dataframe):
    words = 0
    for comment in dataframe["comment_text"]:
        words += len(comment)
    
    return round(words / len(dataframe))


print("Avarage ammount words per comment")
print("=======")
print("Clean docs: \t", calculate_avarage_words(cleant_documents))
print("=======")
print("Toxic docs: \t", calculate_avarage_words(has_toxic))
print("Severe Toxic: \t", calculate_avarage_words(has_severe_toxic))
print("Obscene docs: \t", calculate_avarage_words(has_obscene))
print("Threat docs: \t", calculate_avarage_words(has_threat))
print("Insult docs: \t", calculate_avarage_words(has_insult))
print("Hate docs: \t", calculate_avarage_words(has_identity_hate))

Avarage ammount words per comment
Clean docs: 	 42
Toxic docs: 	 35
Severe Toxic: 	 63
Obscene docs: 	 35
Threat docs: 	 42
Insult docs: 	 34
Hate docs: 	 38


In [ ]:
cleant_documents

In [80]:
insult_unigram = make_grams(cleant_documents)

insult_unigram.most_common(10)

[(("''",), 224642),
 (('i',), 170184),
 (('``',), 147895),
 (('',), 83433),
 (('article',), 53756),
 (('the',), 43227),
 (('page',), 43177),
 (('wikipedia',), 39304),
 (('talk',), 33660),
 (('please',), 28601)]

In [ ]:
import numpy as np
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
train = pd.read_csv('train.csv', index_col=0)

X = train.iloc[:, 0]
Y = train.iloc[:, 1:]

In [ ]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

training_data = [ X[0].split() ]
word_to_ix = {}
for sent in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)


In [ ]:
X[0]